##### Summarization Demo

In [ ]:
pip install cohere altair umap-learn -q

In [ ]:
import cohere
import time
import pandas as pd

api_key = 'A5aqcw7FeLSL5sqynr0F2NMlZ8S7P9zQvFaeUJs0'
co = cohere.Client(api_key)


In [ ]:
prompt = '''"The killer whale or orca (Orcinus orca) is a toothed whale belonging to the oceanic dolphin family, of which it is the largest member"
In summary: "The killer whale or orca is the largest type of dolphin"
---
"It is recognizable by its black-and-white patterned body"
In summary:"Its body has a black and white pattern"
---
"Killer whales have a diverse diet, although individual populations often specialize in particular types of prey"
In summary:"'''

print(prompt)

"The killer whale or orca (Orcinus orca) is a toothed whale belonging to the oceanic dolphin family, of which it is the largest member"
In summary: "The killer whale or orca is the largest type of dolphin"
---
"It is recognizable by its black-and-white patterned body"
In summary:"Its body has a black and white pattern"
---
"Killer whales have a diverse diet, although individual populations often specialize in particular types of prey"
In summary:"


In [29]:
n_generations = 5

prediction = co.generate(
    model = "command",
    prompt = prompt,
    return_likelihoods = 'GENERATION',
    stop_sequences = ['"'],
    max_tokens = 60,
    temperature = 0.7,
    num_generations = n_generations,
    k=0,
    p=0.75
)

In [30]:
gens = []
likelihoods = []

for gen in prediction.generations :
  gens.append(gen.text)

  sum_likelihood = 0
  for t in gen.token_likelihoods :
    sum_likelihood += t.likelihood

  likelihoods.append(sum_likelihood)


In [31]:
pd.options.display.max_colwidth = 200

df = pd.DataFrame({'generation' : gens, 'likelihood' : likelihoods})

print(df)

                                                                                                      generation  \
0   Killer whales have a diverse diet, and the types of prey they consume depend on the individual populations.    
1    The diet of the killer whale tends to vary, with different populations focusing on specific types of prey.    
2    The diet of the killer whale tends to vary, with different populations focusing on specific types of prey.    
3    The diet of the killer whale tends to vary, with different populations focusing on specific types of prey.    
4        Killer whales eat different types of prey, but individual populations often specialize in certain prey.   

   likelihood  
0   -6.087838  
1   -3.140059  
2   -3.140059  
3   -3.140059  
4   -5.141917  


In [32]:
df = df.drop_duplicates(subset=['generation'])

print(df)

                                                                                                      generation  \
0   Killer whales have a diverse diet, and the types of prey they consume depend on the individual populations.    
1    The diet of the killer whale tends to vary, with different populations focusing on specific types of prey.    
4        Killer whales eat different types of prey, but individual populations often specialize in certain prey.   

   likelihood  
0   -6.087838  
1   -3.140059  
4   -5.141917  


In [27]:
df = df.sort_values('likelihood', ascending=False, ignore_index=True)

df

,generation,likelihood
0,"Killer whales have a diverse diet, with different populations specializing in specific types of prey.",-3.698776
1,"Killer whales eat different types of prey, but individual populations often stick to specific menus.",-5.580900
2,"The diet of the killer whale tends to vary, with different populations specializing in specific types of food.",-5.731553


##### Langchain Demo

In [ ]:
pip install langchain
pip install qdrant-client

In [54]:
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain.vectorstores import Qdrant
from langchain.document_loaders import TextLoader

import textwrap as tr

import random
import dotenv
import os

dotenv.load_dotenv("/content/sample_data/.env")


True

In [5]:
import tensorflow_datasets as tfds

dataset = tfds.load("trec", split="train")
texts = [item['text'].decode('utf-8') for item in tfds.as_numpy(dataset)]
print(f"Number of Documents : {len(texts)}")

type(texts)

list

In [56]:
random.seed(11)

for item in random.sample(texts,5) :
  print (item)

What is the starting salary for beginning lawyers ?
Where did Bill Gates go to college ?
What task does the Bouvier breed of dog perform ?
What are the top boy names in the U.S. ?
What is a female rabbit called ?


In [78]:
embeddings = CohereEmbeddings(model = "multilingual-22-12")

db = Qdrant.from_texts(texts, embeddings, location=":memory:", collection_name="my_documents", distance_func="Dot")

In [79]:
queries =["How to get in touch with Bill Gates" ,
          "Comment entrer en contact avec Bill Gates",
          "Cara menghubungi Bill Gates"]
queries_lang = ["English", "French", "Indonesian"]


In [80]:
answers = []

for query in queries :
  docs = db.similarity_search(query)
  answers.append(docs[0].page_content)

print(answers)

['What is Bill Gates of Microsoft E-mail address ?', 'What is Bill Gates of Microsoft E-mail address ?', 'What is Bill Gates of Microsoft E-mail address ?']


In [81]:
for idx, query in enumerate(queries) :
  print(f"Query Language: {queries_lang[idx]}")
  print(f"Query : {query}")
  print(f"Most similar Question present in Documents : {answers[idx]}")
  print("*"*20)

Query Language: English
Query : How to get in touch with Bill Gates
Most similar Question present in Documents : What is Bill Gates of Microsoft E-mail address ?
********************
Query Language: French
Query : Comment entrer en contact avec Bill Gates
Most similar Question present in Documents : What is Bill Gates of Microsoft E-mail address ?
********************
Query Language: Indonesian
Query : Cara menghubungi Bill Gates
Most similar Question present in Documents : What is Bill Gates of Microsoft E-mail address ?
********************


##### Prompt Engineering Demo

In [84]:
import cohere
import pandas as pd
import numpy as np
import altair as alt
import textwrap as tr

In [85]:
co = cohere.Client('A5aqcw7FeLSL5sqynr0F2NMlZ8S7P9zQvFaeUJs0')

def generate_text(prompt, temp=0) :

  response = co.generate(
      model = "command",
      prompt = prompt,
      max_tokens = 500,
      temperature = temp
      )
  return response.generations[0].text

In [86]:
user_input = "a wireless headphone product named the CO-1T"
prompt = f"""Write a creative product description for {user_input}"""

response = generate_text(prompt, temp=0.5)
print(response)

 The CO-1T is a revolutionary new wireless headphone that's taking the audio world by storm. Its sleek and stylish design is just the start - these headphones are packing some serious tech under the hood.

The CO-1T boasts an impressive 30 hours of battery life on a single charge, so you can keep listening all day long. And when you finally do run out of juice, charge them up for just 10 minutes and you've got another 12 hours of listening time! They also come with a handy charging case that's perfect for on-the-go charging. 

But the real magic happens when you put them on. The CO-1T's proprietary sound technology ensures crystal-clear audio with an immersive soundstage that will truly transport you to the recording studio with the artist. Every detail of the song, from the reverberating drums to the subtle whispers in the background, is delivered with impeccable clarity and precision. 

And if you're looking for a personalized listening experience, the CO-1T's adjustable noise-cancel

In [90]:
#Specifics

user_input_product = "a wireless headphone product named the CO-1T"
user_input_keywords = '"bluetooth", "wireless", "fast charging"'
user_input_customer = "a software developer who works in noisy offices"
user_input_describe = "benefits of this product"

prompt = f"""Write a creative product description for {user_input_product} with the keywords {user_input_keywords}
for {user_input_customer} and describe the {user_input_describe}"""

response = generate_text(prompt, temp=0.5)
print(tr.fill(response, width=100))




 Are you a software developer struggling to concentrate in the midst of your loud and bustling
workplace? Fear not, for the CO-1T wireless headphones are here to save the day (and your
productivity)! With its sleek design and state-of-the-art technology, these headphones are the
perfect addition to any tech-savvy developer's toolkit.   Crafted with the latest Bluetooth
technology, the CO-1T seamlessly connects to your devices, keeping you wire-free and effortlessly
immersed in your coding world. Its wireless design guarantees maximum mobility, freeing you to move
around as you please without compromising the beat of your productivity drum.   But what happens
when the battery runs low and your coding symphony is threatened? This is where the CO-1T's fast
charging feature truly shines. With just a quick recharge, it grants you hours of uninterrupted
listening, reducing the time spent fiddling with wires or waiting for your headphones to charge.
So why just imagine a peaceful coding envir

In [91]:
#Structured

user_input_product = "a wireless headphone product named the CO-1T"
user_input_keywords = '"bluetooth", "wireless", "fast charging"'
user_input_customer = "a software developer who works in noisy offices"
user_input_describe = "benefits of this product"

prompt = f"""Write a creative product description for {user_input_product}
Keywords : {user_input_keywords}
Audience : {user_input_customer}
Describe :  {user_input_describe}"""

response = generate_text(prompt, temp=0.5)
print(tr.fill(response, width=100))




 Do you hate working in a noisy environment and getting distracted in the middle of your code? Well,
this headache ends now, with our brand new CO-1T wireless headphones.   With the CO-1T headphones,
you can connect to your Bluetooth-enabled devices and immerse yourself in your tasks without
distractions. The headphones are engineered to provide a comfortable listening experience, so you
can focus on what matters.   The CO-1T headphones come with a fast-charging feature, so you won't
have to worry about battery life. It provides you with a convenient and hassle-free experience so
you can focus on what matters.   Say goodbye to distractions and hello to productivity with the
CO-1T wireless headphones.


In [92]:
#Without Context


user_input = "What are the key features of the CO-1T Bluetooth Wireless headphones?"
prompt = user_input

response = generate_text(prompt, temp=0.5)
print(tr.fill(response, width=100))

 The CO-1T Bluetooth Wireless Headphones feature Active Noise Cancellation, Transparent Hearing, and
an Anti-Wind Noise Algorithm to provide a versatile on-the-go listening experience. They are
equipped with three microphones on each earcup, one facing outward towards the environment and one
inward towards the ear, with the third situated at the mouth for calls. These work in conjunction
with an intelligent ANC algorithm to block out distracting noises at up to 38dB attenuation, while
allowing you to tune into important announcements or conversations by switching on Transparent
Hearing.   They have a sleek and stylish design, with premium craftsmanship and ergonomic ear
cushions made from breathable materials, for maximum comfort and superior sound isolation. The CO-1T
headphones have a battery life of up to 35 hours in ANC mode and up to 45 hours in Bluetooth mode,
which is ideal for long-haul flights and all-day excursions. They also support fast charging, with
just a 10-minute charg

In [93]:
#With Context

context = """Think back to the last time you were working without any distractions in the office. That's right...I bet it's been a while. \
With the newly improved CO-1T noise-cancelling Bluetooth headphones, you can work in peace all day. Designed in partnership with \
software developers who work around the mayhem of tech startups, these headphones are finally the break you've been waiting for. With \
fast charging capacity and wireless Bluetooth connectivity, the CO-1T is the easy breezy way to get through your day without being \
overwhelmed by the chaos of the world."""


user_input = "What are the key features of the CO-1T Bluetooth Wireless headphones"

prompt = f"""Given the information above, answer this question: {user_input}"""

response = generate_text(prompt, temp=0)
print(tr.fill(response, width=100))

 The CO-1T Bluetooth Wireless Headphones feature Active Noise Cancellation and an ergonomic design,
and are said to deliver a superior sound quality, even while working out.   They are also water-
repellent and have an impressive battery life. Additionally, they feature an innovative Bluetooth
5.3 technology for enhanced connectivity.   The headphones also have an in-built equalizer that
allows users to customize the sound to their preference, providing an immersive listening
experience.


In [95]:
#Without Examples

prompt="""Turn the following message to a virtual assistant into the correct action:
Send a message to Alison to ask if she can pick me up tonight to go to the concert together"""

response = generate_text(prompt, temp=0)
print(tr.fill(response, width=100))

 Ok, I will send a message to Alison asking if she can pick you up tonight to go to the concert
together. Please provide me with the details of your inquiry so I can assist you better!


In [102]:
#With Examples

user_input = "Send a message to Alison to ask if she can pick me up tonight to go to the concert together"
# user_input = "Ask Alison if she can pick me up tonight to go to the concert together"

prompt=f"""Turn the following message to a virtual assistant into the correct action:

Message: Ask my aunt if she can go to the JDRF Walk with me October 6th
Action: can you go to the jdrf walk with me october 6th

Message: Ask Eliza what should I bring to the wedding tomorrow
Action: what should I bring to the wedding tomorrow

Message: Send message to supervisor that I am sick and will not be in today
Action: I am sick and will not be in today

Message: {user_input}
Action:"""

response = generate_text(prompt, temp=0.5)
print(tr.fill(response, width=100))

 can you pick me up tonight to go to the concert together?


In [117]:
#Chain of thought

prompt=f"""
Q: Roger has 5 tennis balls. He buys 6 more tennis balls. How many tennis balls does he have now?
A: The answer is 11.
Q: The cafeteria had 23 apples. If they ate 20 for lunch and later bought 6 apples, how many apples do they have now?
A:"""

response = generate_text(prompt, temp=0)
print(tr.fill(response, width=100))

 The cafeteria now has 29 apples. They started with 23 apples, ate 20 at lunch, and then bought 6
more, so they have 29 total apples now.


In [114]:
#Tabular Format

prompt="""Turn the following information into a table with columns Invoice Number, Merchant Name, and Account Number.
Bank Invoice: INVOICE #0521 MERCHANT ALLBIRDS ACC XXX3846
Bank Invoice: INVOICE #6781 MERCHANT SHOPPERS ACC XXX9877
Bank Invoice: INVOICE #0777 MERCHANT CN TOWER ACC XXX3846
"""

response = generate_text(prompt, temp=0)
print(response)

 Here is a table formatted as you requested:

| Invoice Number | Merchant Name | Account Number |
| --- | --- | --- |
| INVOICE #0521 | ALLBIRDS | XXX3846 |
| INVOICE #6781 | SHOPPERS | XXX9877 |
| INVOICE #0777 | CN TOWER | XXX3846 |

Let me know if you would like me to make any further alterations to this data set. 


In [116]:
#JSON Format

prompt="""Turn the following information into a JSON string with the following keys: Invoice Number, Merchant Name, and Account Number.
Bank Invoice: INVOICE #0521 MERCHANT ALLBIRDS ACC XXX3846
Bank Invoice: INVOICE #6781 MERCHANT SHOPPERS ACC XXX9877
Bank Invoice: INVOICE #0777 MERCHANT CN TOWER ACC XXX3846
"""

response = generate_text(prompt, temp=0)
print(response)

 Certainly, here is the information provided in a JSON format:
```json
[
    {
        "invoice_number": "INVOICE #0521",
        "merchant_name": "ALLBIRDS",
        "account_number": "XXX3846"
    },
    {
        "invoice_number": "INVOICE #6781",
        "merchant_name": "SHOPPERS",
        "account_number": "XXX9877"
    },
    {
        "invoice_number": "INVOICE #0777",
        "merchant_name": "CN TOWER",
        "account_number": "XXX3846"
    }
]
```


In [121]:
#With Steps

user_input = "education"

prompt = f"""Generate a startup idea for this industry: {user_input}
First, describe the problem to be solved.
Next, describe the target audience of this startup idea.
Next, describe the startup idea and how it solves the problem for the target audience.
Next, provide a name for the given startup.

Use the following format:
Industry: <the given industry>
The Problem: <the given problem>
Audience: <the given target audience>
Startup Idea: <the given idea>
Startup Name: <the given name>"""


response = generate_text(prompt, temp=0.9)
print(response)

 Industry: Education
The Problem: Improving student engagement and participation in online classes
Audience: Secondary and post-secondary students aged 15-24 years
Startup Idea: Intellecto, an AI-assistant designed to foster a fun learning environment, offering interactive ways of learning and engaging with educational materials and peers.
Startup Name: Intellecto


In [122]:
prompt=f"""Give me a step by step calculation for the below question
The cafeteria had 23 apples. If they ate 20 for lunch and later bought 6 apples, how many apples do they have now?
"""

response = generate_text(prompt, temp=0)
print(tr.fill(response, width=100))

 The cafeteria started with 23 apples, consumed 20 for lunch, and bought 6 afterwards. To calculate
the total number of apples they have now, we follow these steps:  1. Start with the initial number
of apples the cafeteria had, which is 23. 2. Subtract the number of apples eaten for lunch, which is
20. 23 - 20 = 3. 3. Add the number of apples they bought later, 6. 3 + 6 = 9.  Therefore, the
cafeteria now has 9 apples.


In [124]:
#Prefix
user_input_position = "modern centre forward"

prompt = f"""Describe the ideal {user_input_position}. In particular, describe the following characteristics: \
pace, skill, and awareness.
"""

response = generate_text(prompt, temp=0.9)
print(response)

 A centre forward is a player that plays in a forward position, often closest to the opposition's goal. 

A centre forward requires many attributes to be successful. Some key characteristics include: 

Pace: The ideal centre forward has tremendous speed and agility, enabling them to accelerate past defenders and stretch the opposition's backline. This pace is crucial for creating space for themselves and their teammates and allows them to be a threat on counter-attacks. 

Skill: Modern centre forwards must possess refined footballing skills, such as dribbling, ball control, and dribbling. These skills allow them to retain possession in dangerous areas, create opportunities for themselves or their teammates, and beat defenders one-on-one. 

Awareness: Centre forwards must also have excellent situational awareness on the field. This allows them to position themselves intelligently within the attacking third, finding pockets of space and making themselves available for passes from teammat